In [1]:
import pandas as pd


In [2]:
df1 = pd.read_csv('train_bodies.csv')
print(df1)
df2= pd.read_csv('train_stances.csv')

print(df2)

      Body ID                                        articleBody
0           0  A small meteorite crashed into a wooded area i...
1           4  Last week we hinted at what was to come as Ebo...
2           5  (NEWSER) – Wonder how long a Quarter Pounder w...
3           6  Posting photos of a gun-toting child online, I...
4           7  At least 25 suspected Boko Haram insurgents we...
...       ...                                                ...
1678     2528  Intelligence agencies hunting for identity of ...
1679     2529  While Daleks "know no fear" and "must not fear...
1680     2530  More than 200 schoolgirls were kidnapped in Ap...
1681     2531  A Guantanamo Bay prisoner released last year a...
1682     2532  ANN ARBOR, Mich. – A pizza delivery man in Mic...

[1683 rows x 2 columns]
                                                Headline  Body ID     Stance
0      Police find mass graves with at least '15 bodi...      712  unrelated
1      Hundreds of Palestinians flee floo

In [3]:
train_stances_id=df2['Body ID']
train_bodies_id=df1['Body ID']
train_data=[]
for each in train_bodies_id:
    if each in train_stances_id:

        target_rows1 = df1[df1['Body ID'] == each]
        target_rows2 = df2[df2['Body ID'] == each]
        for index1, row1 in target_rows1.iterrows():
            for index2, row2 in target_rows2.iterrows():
                train_data.append([row1["Body ID"],row1['articleBody'],row2['Body ID'],row2['Headline'],row2['Stance']])
                

for i in range(20):
    print(train_data[i][0],train_data[i][2],train_data[i][3],train_data[i][4])
                

0 0 Soldier shot, Parliament locked down after gunfire erupts at war memorial unrelated
0 0 Tourist dubbed ‘Spider Man’ after spider burrows under skin for days unrelated
0 0 Luke Somers 'killed in failed rescue attempt in Yemen' unrelated
0 0 BREAKING: Soldier shot at War Memorial in Ottawa unrelated
0 0 Giant 8ft 9in catfish weighing 19 stone caught in Italy is thought to be the biggest ever reeled in with a rod and line unrelated
0 0 Enormous 20-stone catfish caught with fishing rod in Italy after 40-minute boat battle unrelated
0 0 Italian catches huge wels catfish; is it a record? unrelated
0 0 Not coming to a store near you: The pumpkin spice condom unrelated
0 0 One gunman killed in shooting on Parliament Hill in Ottawa, hunt on for other shooters unrelated
0 0 Soldier shot at war memorial in Canada unrelated
0 0 Surreal Photos of Fisherman’s Jaw-Dropping Catch Will Likely Have People Wondering If It’s Real unrelated
0 0 Fisherman lands 19 STONE catfish which could be biggest in

In [4]:
df=pd.DataFrame(train_data,columns=['bodies Body ID','body','Stances Body ID','headline','Stance'])
print(df)



       bodies Body ID                                               body  \
0                   0  A small meteorite crashed into a wooded area i...   
1                   0  A small meteorite crashed into a wooded area i...   
2                   0  A small meteorite crashed into a wooded area i...   
3                   0  A small meteorite crashed into a wooded area i...   
4                   0  A small meteorite crashed into a wooded area i...   
...               ...                                                ...   
49967            2532  ANN ARBOR, Mich. – A pizza delivery man in Mic...   
49968            2532  ANN ARBOR, Mich. – A pizza delivery man in Mic...   
49969            2532  ANN ARBOR, Mich. – A pizza delivery man in Mic...   
49970            2532  ANN ARBOR, Mich. – A pizza delivery man in Mic...   
49971            2532  ANN ARBOR, Mich. – A pizza delivery man in Mic...   

       Stances Body ID                                           headline  \
0         

In [5]:
import tensorflow as tf
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [6]:
# Define the functions to compute external features
s1=df.loc[0,'body']
s2=df.loc[0,'headline']
print(df.loc[0,'Stance'])

def ngram_match(s1, s2, n):
    """Compute the number of n-grams matched between two strings"""
    ngrams1 = set([s1[i:i+n] for i in range(len(s1)-n+1)])
    ngrams2 = set([s2[i:i+n] for i in range(len(s2)-n+1)])
    return len(ngrams1.intersection(ngrams2))

def char_ngram_match(s1, s2):
    """Compute the number of character n-grams matched between two strings"""
    match = []
    for n in range(2, 17):
        match .append(ngram_match(s1, s2, n))
    return match
print(char_ngram_match(s1,s2))
print(len(char_ngram_match(s1,s2)))

def word_ngram_match(s1, s2):
    """Compute the number of word n-grams matched between two strings"""
    match = []
    for n in range(2, 7):
        s1_tokens = word_tokenize(s1)
        s2_tokens = word_tokenize(s2)
        s1_ngrams = set([' '.join(s1_tokens[i:i+n]) for i in range(len(s1_tokens)-n+1)])
        s2_ngrams = set([' '.join(s2_tokens[i:i+n]) for i in range(len(s2_tokens)-n+1)])
        match.append(len(s1_ngrams.intersection(s2_ngrams)))
    return match

print(word_ngram_match(s1,s2))
print(len(word_ngram_match(s1,s2)))
def tfidf_score(s1, s2):
    """Compute the weighted TF-IDF score between two strings"""
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf = vectorizer.fit_transform([s1, s2])
    return (tfidf * tfidf.T).A[0,1]
print(tfidf_score(s1,s2))

def sentiment_difference(s1, s2):
    """Compute the sentiment difference between two strings"""
    sid = SentimentIntensityAnalyzer()
    s1_sentiment = sid.polarity_scores(s1)['compound']
    s2_sentiment = sid.polarity_scores(s2)['compound']
    return abs(s1_sentiment - s2_sentiment)

print(sentiment_difference(s1,s2))
external_feature=char_ngram_match(s1,s2)+word_ngram_match(s1,s2)+[tfidf_score(s1,s2)]+[sentiment_difference(s1,s2)]
print(external_feature)


unrelated
[52, 32, 10, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
15
[0, 0, 0, 0, 0]
5
0.0
0.20589999999999997
[52, 32, 10, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.20589999999999997]


In [6]:
import copy
y = copy.deepcopy(df['Stance'])
print(y)
print(set(y))
for index, value in y.iteritems():
    if value=="unrelated":
        y[index]=[0,0,1,0]
    elif value=="disagree":
        y[index]=[0,1,0,0]
    elif value=="discuss":
        y[index]=[1,0,0,0]
    elif value=="agree":
        y[index]=[0,0,0,1]
print(y)

0        unrelated
1        unrelated
2        unrelated
3        unrelated
4        unrelated
           ...    
49967        agree
49968        agree
49969        agree
49970        agree
49971        agree
Name: Stance, Length: 49972, dtype: object
{'agree', 'discuss', 'unrelated', 'disagree'}
0        [0, 0, 1, 0]
1        [0, 0, 1, 0]
2        [0, 0, 1, 0]
3        [0, 0, 1, 0]
4        [0, 0, 1, 0]
             ...     
49967    [0, 0, 0, 1]
49968    [0, 0, 0, 1]
49969    [0, 0, 0, 1]
49970    [0, 0, 0, 1]
49971    [0, 0, 0, 1]
Name: Stance, Length: 49972, dtype: object


In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Define the functions to compute external features
def ngram_match(s1, s2, n):
    """Compute the number of n-grams matched between two strings"""
    ngrams1 = set([s1[i:i+n] for i in range(len(s1)-n+1)])
    ngrams2 = set([s2[i:i+n] for i in range(len(s2)-n+1)])
    return len(ngrams1.intersection(ngrams2))

def char_ngram_match(s1, s2):
    """Compute the number of character n-grams matched between two strings"""
    match = []
    for n in range(2, 17):
        match .append(ngram_match(s1, s2, n))
    return match

def word_ngram_match(s1, s2):
    """Compute the number of word n-grams matched between two strings"""
    match = []
    for n in range(2, 7):
        s1_tokens = word_tokenize(s1)
        s2_tokens = word_tokenize(s2)
        s1_ngrams = set([' '.join(s1_tokens[i:i+n]) for i in range(len(s1_tokens)-n+1)])
        s2_ngrams = set([' '.join(s2_tokens[i:i+n]) for i in range(len(s2_tokens)-n+1)])
        match.append(len(s1_ngrams.intersection(s2_ngrams)))
    return match

def tfidf_score(s1, s2):
    """Compute the weighted TF-IDF score between two strings"""
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf = vectorizer.fit_transform([s1, s2])
    return (tfidf * tfidf.T).A[0,1]

def sentiment_difference(s1, s2):
    """Compute the sentiment difference between two strings"""
    sid = SentimentIntensityAnalyzer()
    s1_sentiment = sid.polarity_scores(s1)['compound']
    s2_sentiment = sid.polarity_scores(s2)['compound']
    return abs(s1_sentiment - s2_sentiment)

# Load the data as pandas dataframe
import pandas as pd


# Compute the external features and store in numpy array
external_features =[]
for i, row in df.iterrows():
    s1 = row['headline']
    s2 = row['body']
    external_feature=char_ngram_match(s1,s2)+word_ngram_match(s1,s2)+[tfidf_score(s1,s2)]+[sentiment_difference(s1,s2)]
    if i %1000==0:
        print("num:{}, external_feature:{}".format(i,external_feature))
    external_features.append(external_feature)
    # You can add more features here as needed



In [56]:
    

print(external_features[:20])
print(len(external_features[0]))



[[52, 32, 10, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.20589999999999997], [43, 28, 10, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.8053], [31, 16, 5, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.13480000000000003], [31, 18, 6, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.20589999999999997], [69, 64, 29, 14, 6, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0073264573132805115, 0.8053], [58, 39, 15, 6, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.42350000000000004], [33, 19, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 1.1235], [39, 29, 18, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.022997222930638134, 0.8053], [56, 32, 14, 6, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.13480000000000003], [32, 21, 7, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.20589999999999997], [57, 25, 9, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.8053], [50, 32, 16, 11, 7, 5, 3, 2, 1, 

In [57]:
df = pd.DataFrame(data=external_features)
df.to_excel('external_features.xlsx', index=False)

In [59]:
print(y)

0        [0, 0, 1, 0]
1        [0, 0, 1, 0]
2        [0, 0, 1, 0]
3        [0, 0, 1, 0]
4        [0, 0, 1, 0]
             ...     
49967    [0, 0, 0, 1]
49968    [0, 0, 0, 1]
49969    [0, 0, 0, 1]
49970    [0, 0, 0, 1]
49971    [0, 0, 0, 1]
Name: Stance, Length: 49972, dtype: object


In [8]:
external_features_df=pd.read_excel('external_features.xlsx')
external_features=external_features_df.values.tolist()


In [17]:
print(external_features[:10])


from sklearn.model_selection import train_test_split
#拆分数据集
x_train,x_test,y_train,y_test=train_test_split(external_features,y,random_state=666)
#特征标准化处理
from sklearn.preprocessing import StandardScaler
std=StandardScaler()
#对训练样本集机型特征标准化处理
x_train_standard=std.fit_transform(x_train)
#对测试样本集进行特征标准化处理，要注意这里不能fit了
#测试集的z—score标准化用的均值和标准差是训练集的，所以不用在fit直接转换就行了
x_test_standard=std.transform(x_test)
print('x_train',x_train[:10])
print(y[:10])

[[52.0, 32.0, 10.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2059], [43.0, 28.0, 10.0, 3.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8053], [31.0, 16.0, 5.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1348], [31.0, 18.0, 6.0, 2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2059], [69.0, 64.0, 29.0, 14.0, 6.0, 3.0, 2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.007326457313280512, 0.8053], [58.0, 39.0, 15.0, 6.0, 2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4235], [33.0, 19.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.1235], [39.0, 29.0, 18.0, 6.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02299722293063813, 0.8053], [56.0, 32.0, 14.0, 6.0, 

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim

# 定义 MLP 模型
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.softmax(out)
        return out

# 定义训练数据和标签
X_train = torch.tensor(x_train_standard, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)

# 定义模型和优化器
model = MLP(input_size=22, hidden_size=64, output_size=4)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# 训练模型
num_epochs = 10
batch_size = 32
for epoch in range(num_epochs):
    for i in range(0, X_train.shape[0], batch_size):
        # 获取 mini-batch
        inputs = X_train[i:i+batch_size]
        labels = y_train[i:i+batch_size]

        # 向前传播和计算损失
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # 向后传播和更新参数
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 每个 epoch 结束后输出一下损失值
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

# 保存模型
torch.save(model.state_dict(), 'mlp_model.pth')

# # 加载模型并预测
# model = MLP(input_size=22, hidden_size=64, output_size=4)
# model.load_state_dict(torch.load('mlp_model.pth'))



C:\Users\hw\AppData\Local\Temp\ipykernel_45940\1176397609.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.float32)


Epoch [1/10], Loss: 0.8885
Epoch [2/10], Loss: 0.8870
Epoch [3/10], Loss: 0.8868
Epoch [4/10], Loss: 0.8867
Epoch [5/10], Loss: 0.8867
Epoch [6/10], Loss: 0.8866
Epoch [7/10], Loss: 0.8866
Epoch [8/10], Loss: 0.8866
Epoch [9/10], Loss: 0.8866
Epoch [10/10], Loss: 0.8866


In [21]:
# 设置模型为评估模式
model.eval()

# 将测试数据转换为张量
X_test = torch.tensor(x_test_standard, dtype=torch.float32)

# 使用模型进行预测
with torch.no_grad():
    y_pred = model(X_test)

# 将预测结果转换为 numpy 数组
y_pred = y_pred.numpy()

# 打印预测结果
print(y_pred)

from sklearn.metrics import accuracy_score, confusion_matrix

# 将 softmax 输出转换为类别标签
y_pred_class = np.argmax(y_pred, axis=1)

# 将 y_test 转换为类别标签（如果 y_test 是 one-hot 编码）
y_test_class = np.argmax(y_test, axis=1)

# 计算准确率和混淆矩阵
accuracy = accuracy_score(y_test_class, y_pred_class)
confusion_mat = confusion_matrix(y_test_class, y_pred_class)

print('Accuracy:', accuracy)
print('Confusion Matrix:')
print(confusion_mat)

[[1.00000000e+00 3.55326721e-16 8.56593984e-13 4.18955461e-11]
 [9.89915788e-01 4.42059718e-06 1.50961450e-05 1.00646978e-02]
 [1.00000000e+00 2.33253439e-13 1.98370678e-11 1.66451863e-09]
 ...
 [2.16790184e-07 4.71871031e-09 9.99999762e-01 2.93728863e-09]
 [1.13218505e-07 5.78832626e-09 9.99999881e-01 1.99991912e-09]
 [1.27802929e-02 2.66522075e-05 9.87134516e-01 5.85276794e-05]]
Accuracy: 0.8848955415032418
Confusion Matrix:
[[2084    0  121    5]
 [ 184    0   13    2]
 [ 162    0 8958    0]
 [ 892    0   59   13]]


In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, concatenate

# Define the input shape for each feature vector
input_shape = (10,)  # Assuming each feature vector has 10 elements

# Define the number of neurons in each MLP layer
mlp_units = 32

# Define the number of classes for classification
num_classes = 4

# Input layers for each feature vector
input_1 = Input(shape=input_shape)
input_2 = Input(shape=input_shape)
input_3 = Input(shape=input_shape)

# MLP layers for each feature vector
mlp_1 = Dense(mlp_units, activation='relu')(input_1)
mlp_2 = Dense(mlp_units, activation='relu')(input_2)
mlp_3 = Dense(mlp_units, activation='relu')(input_3)

# Concatenate the outputs of the MLP layers
concatenated = concatenate([mlp_1, mlp_2, mlp_3])

# MLP layer for the concatenated output
output = Dense(num_classes, activation='softmax')(concatenated)

# Create the model
model = Model(inputs=[input_1, input_2, input_3], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Generate random data for demonstration
# Replace this with your actual data
x1 = np.random.rand(100, 10)
x2 = np.random.rand(100, 10)
x3 = np.random.rand(100, 10)
y = np.random.randint(num_classes, size=(100,))

# Convert the target labels to one-hot encoding
y_one_hot = np.eye(num_classes)[y]

# Train the model
model.fit([x1, x2, x3], y_one_hot, epochs=10, batch_size=32)

# Make predictions on new data
# Replace this with your actual test data
x1_test = np.random.rand(10, 10)
x2_test = np.random.rand(10, 10)
x3_test = np.random.rand(10, 10)

predictions = model.predict([x1_test, x2_test, x3_test])
print(predictions)
